In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import matplotlib.style as style
import cv2
import time
import os
from utility.utils import *
import re
import h5py
import tables

style.use('seaborn-paper')
%matplotlib tk


R_m = 1737.1 # lunar radius 
deg2km = 2*np.pi*R_m/360 # deg to km



def compute_K_vet(triplet):
    a,b,c = compute_sides(triplet)
    A,B,C = findAngles(a,b,c)
    K_vet = set([A,B,C])
    if K_vet is not None:
        return K_vet


def compute_sides(triplet):
    a = np.linalg.norm(triplet[0][0:2]-triplet[1][0:2])
    b = np.linalg.norm(triplet[1][0:2]-triplet[2][0:2])
    c = np.linalg.norm(triplet[2][0:2]-triplet[0][0:2])
    return a,b,c



def find_triplet( df_sort_f, index: int ):
    # Input: Pandas DataFrame Sorted i,j
    # Output: Triplet for main crater idx

    def ij_pick(M, k):
        return M[(M.j == j-k) | (M.j == j+k) | (M.i == i+k) | (M.i == i - k)]

    M1 = df_sort_f
    pick1 = M1.iloc[index]
    i = int(pick1.i)
    j = int(pick1.j)
    
    HP, ind = np.zeros(2), 0

    for idx in range(30):
        if idx == 0:
            tmp = ij_pick(M1, idx+1)
            PICKS = tmp
        else:
            tmp = ij_pick(M1, idx+1)
            PICKS = pd.concat([PICKS,tmp])

    for pick in PICKS.iloc:
        pick2 = pick
        dist_12 = np.linalg.norm(pick2[0:2]-pick1[0:2]) # Lat-Lon diff.
        if dist_12*deg2km > pick2.r + pick1.r: # crater condition: dist > r1 + r2
            hp = np.hstack([ind, dist_12])  
            HP = np.vstack([HP, hp])
        ind +=1
    # CHECK:
    if len(HP) < 4: return None
    else:
        HP = HP[1:,:].copy() # Remove first zeros
        HP.view('f8,f8').sort(order=['f1'], axis=0) # Order by Eu Distance
        c1 = pick1
        c2 = PICKS.iloc[int(HP[0,0])]
        c3 = PICKS.iloc[int(HP[1,0])]
        return c1,c2,c3, PICKS

def dist_crt( c1, c2 ):
    # Input: crater 1 and crater 2
    # Output: distance in Km
    return np.linalg.norm(c1[0:2]-c2[0:2])*deg2km


def find_triplets(df):
    N, triplet_list = len(df), []
    for idx in range(N):
        # 1st:
        if idx == 0:
            c1,c2,c3 = find_triplet(df, 0)
            if np.all([c1,c2,c3]) != None: # If they exist
                entry = set([c1.name, c2.name, c3.name])
                triplet_list.append(entry)
        # 2nd to end:
        else:
            c1,c2,c3 = find_triplet(df, idx)
            if np.all([c1,c2,c3]) != None: # If they exist
                entry = set([c1.name, c2.name, c3.name]) # Note: set() make order meaningless
                if entry not in triplet_list:
                    triplet_list.append(entry)
        printProgressBar(idx, N, prefix="Progress:", suffix="Complete", length=50)

    return triplet_list

def picktrip(TRI, idx):
    triplet, IDs = TRI[idx], []
    
    for index in triplet: IDs.append(index)
    
    crat1 = craters_det_sort.iloc[IDs[0]]
    crat2 = craters_det_sort.iloc[IDs[1]]
    crat3 = craters_det_sort.iloc[IDs[2]]
    return crat1, crat2,crat3



def findAngles( a, b, c): 
    # applied cosine rule 
    A = np.arccos((b * b + c * c - a * a) / 
                              (2 * b * c)) 
    B = np.arccos((a * a + c * c - b * b) / 
                              (2 * a * c)) 
    C = np.arccos((b * b + a * a - c * c) / 
                              (2 * b * a)) 
    # convert into degrees 
    A, B, C = np.rad2deg(A),np.rad2deg(B),np.rad2deg(C)
    return A,B,C 

In [2]:
filepath= 'DATA/lunar_crater_database_robbins_2018.csv'
df = pd.read_csv(filepath, sep=',')
df = df[ ['LON_CIRC_IMG','LAT_CIRC_IMG','DIAM_CIRC_IMG']  ] # Select columns
df['LON_CIRC_IMG'] = np.where(df['LON_CIRC_IMG'] > 180, df['LON_CIRC_IMG'] - 360, df['LON_CIRC_IMG']) 
df['DIAM_CIRC_IMG'] = df['DIAM_CIRC_IMG'] 

In [3]:
df.rename(columns={'LON_CIRC_IMG':'lon','LAT_CIRC_IMG':'lat','DIAM_CIRC_IMG':'r'}, inplace=True)

sort_lat = df.sort_values(by='lat')
sort_lat.insert(3, 'i', range(len(sort_lat)))

sort_lat_lon = sort_lat.sort_values(by='lon')
sort_lat_lon.insert(4,'j', range(len(sort_lat_lon)))

df_sort = sort_lat_lon.sort_index()

In [4]:
# height_filter_catalog
H = 50
FOV_half = np.deg2rad(90/2)
d = 2*H* np.tan(FOV_half)
filter = 2/3*d

df_sort_f = df_sort[df_sort['r'] < filter].copy()
df_sort_f = df_sort_f.reset_index(drop=True)

In [59]:
crt1, crt2, crt3, PICKS = find_triplet( df_sort_f, 0)
max_crt_dist = np.max([dist_crt(crt1,crt2),dist_crt(crt2,crt3),dist_crt(crt1,crt3)])
# print(max_crt_dist) # Into the query search: filter all triplet with max_crt_dist > d

triplet = [crt1, crt2, crt3]
K_vet = compute_K_vet(triplet)
print(K_vet)
Lat_bounds = [np.min([crt1.lat,crt2.lat,crt3.lat]),  np.max([crt1.lat,crt2.lat,crt3.lat]) ]
Lon_bounds = [np.min([crt1.lon,crt2.lon,crt3.lon]),  np.max([crt1.lon,crt2.lon,crt3.lon]) ]
entry = [ Lat_bounds, Lon_bounds, K_vet, [crt1, crt2, crt3]]
print(entry)


{89.73482510910853, 44.69529589658786, 45.569878994303615}
[[61.0386, 61.6197], [-171.97400000000002, -171.38400000000001], {89.73482510910853, 44.69529589658786, 45.569878994303615}, [lon   -1.719740e+02
lat    6.103860e+01
r      1.352130e+01
i      1.169582e+06
j      3.490800e+04
Name: 0, dtype: float64, lon   -1.719720e+02
lat    6.161970e+01
r      1.005970e+00
i      1.172669e+06
j      3.491700e+04
Name: 2496, dtype: float64, lon   -1.713840e+02
lat    6.103930e+01
r      1.650710e+00
i      1.169586e+06
j      3.756400e+04
Name: 2519, dtype: float64]]


In [73]:
# Crater_TRN_catalog = pd.DataFrame({
# 'Lat_bounds': Lat_bounds,
# 'Lon_bounds': Lon_bounds,
# 'Max ij-crt distance': max_crt_dist,
# })

In [79]:
entry_pd = pd.Series(entry)

In [80]:
Crater_TRN_catalog = pd.DataFrame({'triplet':entry_pd})

In [81]:
Crater_TRN_catalog

,triplet
0,"[61.0386, 61.6197]"
1,"[-171.97400000000002, -171.38400000000001]"
2,"{89.73482510910853, 44.69529589658786, 45.5698..."
3,"[[-171.97400000000002, 61.0386, 13.5213, 11695..."


In [ ]:
# def image_gen(PATH):
PATH = 'DATA/ephemeris sat/20 step size/IMG_20_Lat_0_Long_-127.683.jpg'
path = '/home/sirbastiano/Desktop/Python Projects/Progetto Tesi/'+PATH

# Actual Position:
num =20
l = -127.683
H = 50
X_c, Y_c = l, 0
# x_f, y_f, z_f = spherical2cartesian(0, crater.Lat, crater.Lon)

# Find reference in catalogs:
dteta = find_dteta(H) # d/R, where d^2 is the footprint
sp = dteta/2 # span
lat_bounds, lon_bounds = np.array([Y_c-sp,Y_c+sp]), np.array([X_c-sp,X_c+sp])
print('*'*35 +'BORDERS'+'*'*35)
print(f'LAT: {lat_bounds}              LON:{lon_bounds}')
print('*'*35 +'*******'+'*'*35)
# 3 crater catalogs: HEAD | LROC | ROBBINS
CATALOGS = ['ROBBINS', 'HEAD', 'LROC']
for DATA in CATALOGS:
    if DATA =='HEAD':
        filepath = '/home/sirbastiano/Desktop/Python Projects/Progetto Tesi/DATA/HeadCraters.csv'
    elif DATA == 'LROC':
        filepath = '/home/sirbastiano/Desktop/Python Projects/Progetto Tesi/DATA/LROCCraters.csv'
    elif DATA == 'ROBBINS':
        filepath= '/home/sirbastiano/Desktop/Python Projects/Progetto Tesi/DATA/lunar_crater_database_robbins_2018.csv'

    DB = pd.read_csv(filepath, sep=',')
    df = CatalogSearch(DB, lat_bounds, lon_bounds, CAT_NAME=DATA)
    if df is None:
        print('No Craters found!')
    else:     
        print(f'Found {df.shape[0]} craters!\n\n', df)
    
    ## DRAWINGS: 
    ## FIG.1
    img = cv2.imread(path)
    plt.ion()
    plt.figure('Tile and Mask -'+DATA, dpi=200)
    plt.subplot(1,2,1)
    plt.imshow(img)
    plt.subplot(1,2,2)
    I = draw_craters(df, lon_bounds, lat_bounds)
    plt.imshow(I)
    plt.show()
    plt.savefig(DATA+str(num)+'_fig-A.png')
    
    ## FIG.2
    image_with_craters = draw_craters_on_image(df,  lon_bounds, lat_bounds, img, u=None)
    plt.ion()
    plt.figure('Image with craters catalogued-'+DATA, dpi=200)
    plt.imshow(image_with_craters)
    plt.show()
    plt.savefig(DATA+str(num)+'_fig-B.png')

plt.close('all')

In [ ]:
PATHS = os.listdir("/home/sirbastiano/Desktop/Python Projects/Progetto Tesi/DATA/ephemeris sat/20 step size")
print(PATHS)